In [1]:
!pip install torch

In [2]:
import os, sys

# Get the absolute path of the notebook's directory
notebook_dir = os.getcwd()

# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Add the parent directory to sys.path so we can import modules
sys.path.append(parent_dir)

# Verify the path
print(f"Added to sys.path: {parent_dir}")

# Add it to sys.path
sys.path.append(parent_dir)

from FlashOperation.Refrig2DrumHeatExConstr import Refrig2DrumConstraintHeatExConstr
from module import *

Added to sys.path: C:\Users\ppromte1\OneDrive - Johns Hopkins\Process Design\Aspen


In [3]:
import numpy as np
import pickle
import os

# Define the file path for saving/loading the data
data_file = "flash_simulation_data_sim_redo2.pkl"

# Define parameter ranges
flash_1_range = np.linspace(1, 20, 10)
flash_2_range = np.linspace(1, 6, 10)

# Check if the data file already exists
if os.path.exists(data_file):
    print("Loading existing data...")
    with open(data_file, "rb") as f:
        data = pickle.load(f)
    assSim = Refrig2DrumConstraintHeatExConstr(AspenFile="../FlashOperation/FlashOperation_HeatExchanger.bkp",
                                               wdpath="../FlashOperation",
                                               visibility=False,
                                               Penalty=1e3)
else:

    print("Generating new data...")
    assSim = Refrig2DrumConstraintHeatExConstr(AspenFile="../FlashOperation/FlashOperation_HeatExchanger.bkp",
                                               wdpath="../FlashOperation",
                                               visibility=False,
                                               Penalty=1e3)

    data = []
    for flash_1 in flash_1_range:
        for flash_2 in flash_2_range:
            x_unflat = assSim.unflatten_params([float(flash_1), float(flash_2)])
            data.append([flash_1, flash_2, assSim.run_obj(x_unflat)])

# Save the data for future use
with open(data_file, "wb") as f:
    pickle.dump(data, f)
print(f"Data saved to {data_file}")

# Data is now available
print("Data loaded, number of entries:", len(data))
data = np.array(data)
print(data)



Generating new data...
The current Directory is :  
C:\Users\ppromte1\OneDrive - Johns Hopkins\Process Design\Aspen\localityaware
The new Directory where you should also have your Aspen file is : 
C:\Users\ppromte1\OneDrive - Johns Hopkins\Process Design\Aspen\FlashOperation
Aspen is active now. If you dont want to see aspen open again take VISIBITLY as False 

Data saved to flash_simulation_data_sim_redo2.pkl
Data loaded, number of entries: 100
[[1.00000000e+00 1.00000000e+00 2.30606519e+04]
 [1.00000000e+00 1.55555556e+00 3.84869866e+04]
 [1.00000000e+00 2.11111111e+00 5.67770989e+04]
 [1.00000000e+00 2.66666667e+00 9.01265477e+05]
 [1.00000000e+00 3.22222222e+00 9.05821870e+05]
 [1.00000000e+00 3.77777778e+00 9.14740296e+05]
 [1.00000000e+00 4.33333333e+00 9.23567279e+05]
 [1.00000000e+00 4.88888889e+00 9.32325530e+05]
 [1.00000000e+00 5.44444444e+00 9.41031034e+05]
 [1.00000000e+00 6.00000000e+00 9.49695193e+05]
 [3.11111111e+00 1.00000000e+00 5.36959950e+04]
 [3.11111111e+00 1.555

In [4]:

# Get the list of all files in the surface2 directory and sort them
surface_dir = "../localityaware/surface_redoflash"
surface_files = sorted([f for f in os.listdir(surface_dir) if f.endswith('.csv')])

# Delete the surface files
for surface_file in surface_files:
    os.remove(os.path.join(surface_dir, surface_file))

In [6]:
# **Initialize Model & Datasets**
old_dataset = OldDataSet(data, k=40)
new_dataset = NewDataSet(k=3)
model = MLP(2, [20, 20, 20], 1)
scaler = MinMax_Scaler((-1,1), 
                       max_vals=[34,34],
                       min_vals =[0,0],
                       min_y=0,
                       max_y = 1e6,
                       scale_y=True)

x_path, y_path = optimize_surrogate_model_new(model, 
                                          old_dataset,
                                          new_dataset, 
                                          assSim, 
                                          optim_steps=100, 
                                          N_s=30, 
                                          lr={"model learning rate": 1e-4, "input learning rate": 1e-2}, 
                                          merge_interval=5,
                                          x_init = torch.tensor([7, 1.2
                                                                 ], dtype=torch.float32, requires_grad=True),
                                            epochs = {"start":500, "during": 100},
                                            batch_size_old = 64,
                                            batch_size_new = 32,
                                            min_vals=scaler.min_x,
                                            max_vals=scaler.max_x,
                                            patience = 10,
                                              tolerance = 100,
                                          scaler= scaler,
                                          log_file = '../../Aspen/localityaware/opt_log_redo.csv',
                                          calls_log_file=  '../../Aspen/localityaware/calls_log_redo.csv',
                                          surface_log_folder = '../../Aspen/localityaware/surface_redoflash',
                                          device='cpu',
                                          lambda_mse=100,
                                          new_data_weight=200,
                                          upsteps=1,
                                          stddev=0.5
                                         )


device cpu
⚠️ No new data available, training only on old dataset.
✅ Neural network surface saved at step 0: ../../Aspen/localityaware/surface_redoflash\surface_step_0.csv
Step 1/100 - Time: 0.00s
device cpu
Inner step 0: x_init = [7.010000228881836, 1.190000057220459]
y_pred = 10629.431640625
grad = [-3777.12255859375, 20998.181640625]
Step 2/100 - Time: 17.49s
device cpu
Inner step 0: x_init = [7.019886493682861, 1.180017352104187]
y_pred = 8478.03125
grad = [-5846.0888671875, 27347.810546875]
Step 3/100 - Time: 35.75s
device cpu
Inner step 0: x_init = [7.029799461364746, 1.1700081825256348]
y_pred = 7310.33984375
grad = [-7076.36669921875, 31012.05078125]
Step 4/100 - Time: 53.66s
device cpu
Inner step 0: x_init = [7.039754867553711, 1.1599631309509277]
y_pred = 6114.318359375
grad = [-8327.1767578125, 34966.515625]
Step 5/100 - Time: 73.32s


com_error: (-2147023170, 'The remote procedure call failed.', None, None)

In [ ]:
import pickle
with open('x_path_redo.pkl', 'wb') as f:
    pickle.dump(x_path, f)
with open('y_path_redo.pkl', 'wb') as f:
    pickle.dump(y_path, f)